# Unveiling Academic and Social Patterns: A Comprehensive Analysis of Student Alcohol Consumption Dataset

### by Brian Adhitya, Wyatt Howell, Brian Ko, Nishanth Selvakumar

### Introduction

Alcohol consumption has been high around the world, especially for students. ...

With this analysis, we are trying to gain insights on what actually causes this students to drink, and ...

### Preeliminary exploratory data analysis

**1. Loading the Dataset**

**2. Analyzing columns and rows**

**3. ...**

### Methods

**1. ...**

### Expected Outcomes and Significance

**1. ...**

### Bibliography

P. Cortez and A. Silva. Using Data Mining to Predict Secondary School Student Performance. In A. Brito and J. Teixeira Eds., Proceedings of 5th FUture BUsiness TEChnology Conference (FUBUTEC 2008) pp. 5-12, Porto, Portugal, April, 2008, EUROSIS, ISBN 978-9077381-39-7.

Fabio Pagnotta, Hossain Mohammad Amran.
Email:fabio.pagnotta@studenti.unicam.it, mohammadamra.hossain '@' studenti.unicam.it
University Of Camerino

https://archive.ics.uci.edu/ml/datasets/STUDENT+ALCOHOL+CONSUMPTION